In [25]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [26]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [27]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_66.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 4
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))



Professor Dagger examines the seance table and the three leather bound books with utmost concentration and scrutiny, being as careful as possible to not inflict any damage to the items.

The Professor, in a somber tone, inquired about a fourth book, which is the rarest of them all, for it allows the user to bind a demon to the material world, to earth. But it would burst in flames after the transaction is complete, unless the Demon has been betrayed and unable to collect its debt. He concludes, altogether, along with the fourth book, these items could fetch a price well above $2,000,000. Your eyes lit up, recognizing this to be your chance to turn your life around, pay off your debts and perhaps start something by yourself.

You informed him that you found all the items in the recently acquired storage room, and suggested to the Professor that the fourth book may be misplaced in the storage. The Professor offered to help you search for the fourth book, hoping to move things along qui

I select option 1. TURN: 1 


As the truck hums along the road, you nod at Professor Dagger, signaling him to proceed with his confession. He clears his throat, the weight of his words seemingly heavy on his shoulders. "Kyle, there's something you must know about the previous owner of the storage you've acquired," he begins, his voice tinged with a seriousness that immediately captures your attention.

"The owner, along with her family, was deeply involved in the occult. It's not just hearsay; there are records, though most are shrouded in secrecy and whispers of dark rituals. The most unsettling part," he pauses, glancing at you to gauge your reaction, "is the legend surrounding the fourth book. They say the owner conducted a forbidden ritual, using the book to summon a demon and bind it to her sister. The aftermath of that night is lost to rumors, but it's believed that what transpired was too horrific for words."

He lets his words hang in the air, allowing you to absorb the gravity


Memories:
"""
- Professor Dagger confesses to Kyle that the previous owner of the storage and her family were deeply involved in the occult, and that a demonic ritual involving the fourth book may have taken place, binding a demon to the owner's sister. The aftermath of the event is shrouded in mystery and horror.
- Kyle Jackson, despite your financial desperation, you accepted a middle-aged woman's request to access the storage room heyou had recently purchased. Upon entering the storage, Kyle discovered strange and unsettling objects, including a collection of old jars with unidentifiable contents, which he failed to appraise. His investigation led him to a hidden compartment containing maritime maps, a rusted compass, and a journal detailing a voyage to the mythical R'lyeh, driving him to the brink of madness. As Kyle loses control, haunted by the implications of his findings, he hears a strange noise, indicating that something else is present in the storage room with him.
"""


Yo

I select option 2. TURN: 1 


As you tread carefully behind Professor Dagger, the foul odor becomes increasingly unbearable, prompting you to ask, "What is that dreadful smell?" 

Professor Dagger pauses for a moment, his lantern casting long shadows on the walls as he turns to face you. "Decay," he replies grimly. "This place has seen death, and what lies at the end might be the source. Be on your guard, Kyle. Not just for what we might find, but for what might find us."

The corridor stretches ahead, its end swallowed by darkness. The air is thick, making each breath a struggle against the invisible weight of the unknown. The warning does little to ease the tension, but it solidifies your resolve to uncover the truth behind the storage's sinister secrets.

As you continue forward, the sense of unease grows with each step, the darkness seeming to pulse with a life of its own. The path you walk feels like it's leading not just to the heart of the storage room, but perhaps to something 

I select option 1. TURN: 2 


You choose to continue following Professor Dagger in silence, letting the gravity of the situation and his previous warning guide your actions. The corridor seems to stretch on indefinitely, with each step echoing ominously off the unseen walls. The air grows colder, and the foul stench intensifies, a constant reminder of the decay that Professor Dagger mentioned.

Finally, the corridor ends, opening into a hidden room that is dimly lit by a single, flickering lantern on a table. The sight before you is unsettling: at the center of the room lies the near withered body of a woman, sprawled across the floor in a grotesque manner. Her face is completely gone, leaving a gaping cavity that seems to pulse with a sinister energy. Surrounding her is a pentagram made of a strange red fluid, and at the far end of the room, you see the fourth book placed on a podium.

Professor Dagger steps forward cautiously, his eyes never leaving the withered body. "This is it," h

I select option 1. TURN: 3 


You steel your resolve and decide to approach the podium to retrieve the book yourself. Each step towards the podium feels heavy, as if the very air around you is thickening, trying to hold you back. The withered body lies ominously still, but you can't shake the feeling that it's watching you, despite the absence of its eyes.

As you reach the podium and lay your hands on the fourth book, a sudden chill envelops the room. The strange red fluid forming the pentagram begins to evaporate and disappear, leaving behind no trace it was ever there.

The moment the fluid vanishes, the withered body convulses violently. Tentacles, dark and writhing, burst forth from the gaping cavity that was once its face. They lash out in a frenzied dance, grabbing hold of Professor Dagger before he can react. His screams fill the room as the tentacles pull him towards the cavity, consuming him in a matter of seconds.

You stand frozen in shock, the book still in your hands, as 

I select option 2. TURN: 4 


With the book clutched tightly in your hands, you turn and sprint towards the exit, the horrifying scene of Professor Dagger's demise etched into your memory. The sound of the withered body stirring to life behind you sends a surge of adrenaline through your veins. You can hear the slithering of tentacles on the stone floor, a cacophony of nightmares chasing you as you navigate back through the dark corridor.

As you emerge from the hidden room, the reality of your situation sets in. The only exit from the storage facility is locked, as per the rules established by your previous decisions. The knowledge weighs heavily on you as you race towards the door, the sounds of the entity's pursuit echoing ominously behind you.

Reaching the door, you remember the middle-aged woman's request and your acceptance of it. This decision now comes back to aid you, as the door is unlocked, a direct result of your earlier actions. Without hesitation, you throw the door open